In [1]:
import spacy
import pandas as pd
from spacy import displacy

In [2]:
 !python -m spacy download en_core_web_trf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.9/237.9 kB 6.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install spacy-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.8/758.8 kB 19.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 94.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 37.1 MB/

In [5]:
nlp_sm = spacy.load("en_core_web_sm")


In [6]:
def read_conll_file(path):
    sentences = []
    current_sentence = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # blank line means end of sentence
                if current_sentence:
                    sentences.append(" ".join([tok for tok, tag in current_sentence]))
                    current_sentence = []
                continue
            parts = line.split()
            if len(parts) >= 2:
                token, tag = parts[0], parts[-1]
                current_sentence.append((token, tag))
        if current_sentence:  
            sentences.append(" ".join([tok for tok, tag in current_sentence]))
    return sentences

In [7]:
train_sentences = read_conll_file("/kaggle/input/conll003-englishversion/train.txt")
valid_sentences = read_conll_file("/kaggle/input/conll003-englishversion/valid.txt")
test_sentences = read_conll_file("/kaggle/input/conll003-englishversion/test.txt")


In [8]:
df = pd.DataFrame({
    "dataset": (["train"] * len(train_sentences)) +
               (["valid"] * len(valid_sentences)) +
               (["test"] * len(test_sentences)),
    "text": train_sentences + valid_sentences + test_sentences
})

In [10]:
def extract_entities(text, nlp_model):
    doc = nlp_model(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [11]:
df["entities_sm"] = df["text"].apply(lambda x: extract_entities(x, nlp_sm))


In [12]:
df.to_csv("CoNLL003_NER_results.csv", index=False)
print("✅ NER results saved to CoNLL003_NER_results.csv")


✅ NER results saved to CoNLL003_NER_results.csv


In [15]:

sample_text = df.loc[8, "text"]
print("\nSample text:\n", sample_text)

print("\nLaunching displaCy visualization...")
doc = nlp_sm(sample_text)
displacy.render(doc, style="ent", jupyter=True)


Sample text:
 He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .

Launching displaCy visualization...
